In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

df = pd.read_csv('Dataset_timeseries.csv')
scaler = MinMaxScaler()
df['SOPAS'] = scaler.fit_transform(df[['SOPAS']])
df['Time (s)'] = scaler.fit_transform(df[['Time (s)']])

X = df[['SOPAS', 'Time (s)']].values
y = df['Number'].values

def create_sequences(data, target, seq_length):
    xs = []
    ys = []
    for i in range(len(data) - seq_length):
        x = data[i:i + seq_length]
        y = target[i + seq_length]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

seq_length = 5
X_seq, y_seq = create_sequences(X, y, seq_length)
X_seq = X_seq.reshape((X_seq.shape[0], X_seq.shape[1], X_seq.shape[2]))

X_train, X_test, y_train, y_test = train_test_split(X_seq, y_seq, test_size=0.2, random_state=42)

In [2]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Attention

num_features = X_seq.shape[2]
input_layer = Input(shape=(seq_length, num_features))
lstm_out = LSTM(50, return_sequences=True)(input_layer)
attention = Attention()([lstm_out, lstm_out])
lstm_out = LSTM(50)(attention)
output_layer = Dense(1, activation='linear')(lstm_out)

model = Model(inputs=input_layer, outputs=output_layer)
model.compile(optimizer='adam', loss='mse')
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 5, 2)]               0         []                            
                                                                                                  
 lstm (LSTM)                 (None, 5, 50)                10600     ['input_1[0][0]']             
                                                                                                  
 attention (Attention)       (None, 5, 50)                0         ['lstm[0][0]',                
                                                                     'lstm[0][0]']                
                                                                                                  
 lstm_1 (LSTM)               (None, 50)                   20200     ['attention[0][0]']       

In [ ]:
history = model.fit(X_train, y_train, epochs= 20, batch_size=16, validation_split=0.2)


Epoch 1/20
8188/8188 [==============================] - 56s 6ms/step - loss: 0.9477 - val_loss: 0.9028
Epoch 2/20
8188/8188 [==============================] - 51s 6ms/step - loss: 0.8125 - val_loss: 0.7967
Epoch 3/20
8188/8188 [==============================] - 56s 7ms/step - loss: 0.7736 - val_loss: 0.7657
Epoch 4/20
8188/8188 [==============================] - 55s 7ms/step - loss: 0.7599 - val_loss: 0.7379
Epoch 5/20
8188/8188 [==============================] - 52s 6ms/step - loss: 0.7529 - val_loss: 0.7590
Epoch 6/20
8188/8188 [==============================] - 51s 6ms/step - loss: 0.7444 - val_loss: 0.7590
Epoch 7/20
8188/8188 [==============================] - 51s 6ms/step - loss: 0.7397 - val_loss: 0.7337
Epoch 8/20
8188/8188 [==============================] - 52s 6ms/step - loss: 0.7353 - val_loss: 0.7358
Epoch 9/20
8188/8188 [==============================] - 51s 6ms/step - loss: 0.7313 - val_loss: 0.7334
Epoch 10/20
8188/8188 [==============================] - 51s 6ms/step - l

In [ ]:
y_pred = model.predict(X_test)

from sklearn.metrics import mean_squared_error
import math
rmse = math.sqrt(mean_squared_error(y_test, y_pred))
print(f'RMSE: {rmse}')

In [17]:
import matplotlib.pyplot as plt

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()


NameError: name 'history' is not defined